In [ ]:
from configure import *
from deeplabV3Model import *
from network_utils import *
import segmentation_models as sm

In [ ]:
def make_predictions(weights,image_patches,model="DeeplabV3+"):

    ## loading the trained model
    if model.lower() =="deeplabv3+": 
        trained_learner = DeeplabV3Plus(image_size=p_s, num_classes=NUM_CLASSES)
        trained_learner.load_weights(weights)
    elif model.lower() == "resunet":
        trained_learner = ResUNet((512, 512, 3))
        trained_learner.load_weights(weights)
    elif model.lower() == "resnet":
        trained_learner = sm.Unet('resnet50', 
                input_shape=(p_s, p_s, 3),
                classes=NUM_CLASSES, activation='sigmoid')
        trained_learner.load_weights(weights)

    # image_patches = get_img_patches(years)
    image_patches = np.squeeze(image_patches, axis=2)
    
    predictions = []
    for i in range(image_patches.shape[0]):
            predictions.append(trained_learner.predict(image_patches[i]))
    return predictions

def create_subgraph_around_terminal(matrix, terminal, radius):
    rows, cols = matrix.shape
    G = nx.DiGraph()
    
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    
    t_x, t_y = terminal
    for i in range(max(0, t_x - radius), min(rows, t_x + radius + 1)):
        for j in range(max(0, t_y - radius), min(cols, t_y + radius + 1)):
             if matrix[i][j] < large_val:
                    G.add_edge((i, j, 1), (i, j, 2), weight=matrix[i][j])
    
    for i in range(max(0, t_x - radius), min(rows, t_x + radius + 1)):
        for j in range(max(0, t_y - radius), min(cols, t_y + radius + 1)):
            if matrix[i][j] < large_val:
                for di, dj in directions:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < rows and 0 <= nj < cols and matrix[i][j] < large_val:
                        G.add_edge((i, j, 2), (ni, nj, 1), weight=0)
                        G.add_edge((ni, nj, 2), (i, j, 1), weight=0)
    
    return G


def find_shortest_path(G, terminal, sources):
    # G = create_transformed_graph(matrix)
    terminal_node = (terminal[0], terminal[1], 2)
    shortest_distance = 999999999
    best_source = None
    best_path = None
    source_nodes = []
    try:
        length, paths = nx.single_source_dijkstra(G, source=terminal_node)
    except nx.NetworkXNoPath:
        print("None")

    path = []
    for source in sources:
        s = (source[0], source[1], 1)
        if s in length and length[s]<shortest_distance:
            shortest_distance = length[s]
            path = paths[s]
            best_source = s
#         if length < shortest_distance:
#             shortest_distance = length
#             best_source = source
#             best_path = path

    return best_source, length, path

In [ ]:
img = np.load(f'/scratch/gza5dr/Canal_Datasets/NHDShape/procssed_data/full_data/processed_images/train_img_set3_2022.npy') 
mask = np.load(f'/scratch/gza5dr/Canal_Datasets/NHDShape/procssed_data/full_data/masks/train_mask_set3_2021.npy') 
p_s = 512


cut_1,cut_2 = img.shape[0]%p_s,img.shape[1]%p_s
img = img[cut_1:,cut_2:,:]
mask = mask[cut_1:,cut_2:]
ground_truth = mask


image_patches = patchify(img,(p_s,p_s,img.shape[2]),step=p_s)
gt_patches = patchify(mask,(p_s,p_s),step=p_s)
                      
weights_I = '/scratch/gza5dr/Current_Canal_Experiments/Canal_Detection_Experiments/Proposed_Model_Pipeline/implementation/framework/final_framework/output_set3/R110_th0.5_rth0.01_4dil_resnet/models/best_it4_R110_th0.5_rth0.01_4dil_resnet.weights.h5'
predictions = make_predictions(weights_I,image_patches,model="resnet")  

In [ ]:
f = np.array(predictions)
f = np.squeeze(f,axis=-1)
canal_mask = ground_truth.copy()
full_predictions_mask_patches = f.copy()

water_mask = ground_truth.copy()
water_mask[water_mask!=2] = 0

canal_mask[canal_mask==3] = 1
canal_mask[canal_mask>1] = 0


full_predictions_mask_patches[full_predictions_mask_patches>=th] = 1
full_predictions_mask_patches[full_predictions_mask_patches<th] = 0

full_predictions_mask = unpatchify(full_predictions_mask_patches, ground_truth.shape) #f


kernel = np.ones((3, 3), np.uint8) 
new_mask = cv2.dilate(canal_mask,kernel=kernel,iterations=5)
new_mask = new_mask + full_predictions_mask
new_mask[new_mask>1] = 1
H = cv2.erode(new_mask, kernel=kernel,iterations=5)
full_h = water_mask+H
full_h[full_h==3]=1
full_h_patches = patchify(full_h,(p_s,p_s),step=p_s)

In [ ]:
def find_common_rows(a, b):
    # Convert b to a set of tuples for faster lookup
    b_set = set(map(tuple, b))
    
    # Use a list comprehension to find common rows while preserving order
    common = [row for row in a if tuple(row) in b_set]
    
    return np.array(common)

gt_patches[gt_patches==3]=1

all_mask = ground_truth.copy()
all_mask[all_mask==3]=1

canal_mask = ground_truth.copy()
canal_mask[canal_mask==3] = 1
canal_mask[canal_mask>1] = 0

###Main GT
water_index = get_water_source_index_patch(all_mask,gt_patches)
directly_connected_patches,not_connected_patches = directly_connected_1s_patch(all_mask,gt_patches,water_index)
reachable_nodes, unreachable_nodes = bfs_patch(gt_patches, directly_connected_patches, directly_connected_patches,not_connected_patches)
print(f"Number of reachable Canal pixels:{len(reachable_nodes)}")

unreachable_nodes = np.where(unreachable_nodes == 1)
i,j,d_row_indices, d_col_indices = unreachable_nodes
unreachable_nodes_index_pairs = list(zip(i,j,d_row_indices, d_col_indices))
unreachable_nodes_index_pairs = np.array(unreachable_nodes_index_pairs)
print(f"Number of unreachable Canal pixels in ground truth: {len(unreachable_nodes_index_pairs)}")

## For H
directly_connected_preds,not_connected_preds = directly_connected_1s_patch(full_h,full_h_patches,water_index)
reachable_nodes_H, unreachable_nodes_H = bfs_patch(full_h_patches, directly_connected_preds, directly_connected_preds,not_connected_preds)
print(f"Number of reachable Canal pixels:{len(reachable_nodes_H)}")


unreachable_nodes_H = np.where(unreachable_nodes_H == 1)
i,j,d_row_indices, d_col_indices = unreachable_nodes_H
unreachable_nodes_index_pairs_H = list(zip(i,j,d_row_indices, d_col_indices))
unreachable_nodes_index_pairs_H = np.array(unreachable_nodes_index_pairs_H)
print(f"Number of unreachable Canal pixels in H: {len(unreachable_nodes_index_pairs_H)}")


a = np.array(unreachable_nodes_index_pairs)
b = np.array(unreachable_nodes_index_pairs_H)

result = find_common_rows(a, b)
print("Number of common canal pixels in GT and H",len(result))

curr_unreachable_nodes = unreachable_nodes_index_pairs
common_unreachable_nodes = result


In [ ]:
def find_isolated_points(data,full_h_patches):
    data_set = set(tuple(point) for point in data)  # Convert array to a set of tuples for fast lookup
    visited = set()  # To track visited points
    endpoints = []  # List to store endpoints
    p_i,p_j,p_rows,p_columns = full_h_patches.shape

    # Directions corresponding to N, NE, E, SE, S, SW, W, NW
    directions = [(0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1)]

    for point in data:
        neighbors = []
        point_tuple = tuple(point)
        i,j,x,y = point_tuple
        if point_tuple not in visited:
            # Check all 8 directions and count neighbors
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                n_i,n_j= i,j

                if 0>nx or nx>=p_rows:
                    n_i = i+dx
                    nx = 511 if nx == -1 else 0 if nx == 512 else nx

                if 0>ny or ny>=p_columns:
                    n_j = j+dy
                    ny = 511 if ny == -1 else 0 if ny == 512 else ny
                    
                if (n_i,n_j,nx, ny) in data_set:
                    neighbors.append((n_i,n_j,nx, ny))

            num_neighbors = len(neighbors)
            
            if num_neighbors <= 1:
                endpoints.append(point)  # Zero or one neighbor, so it's an endpoint
                
            # Mark point and its neighbors as visited
            visited.add(point_tuple)
            # visited.update(neighbors)

    return np.array(endpoints),visited

data = np.array(curr_unreachable_nodes)

terminals,visited = find_isolated_points(data,gt_patches)
print(len(terminals))


unique_patch_nodes = set()

for i,j,_,_ in curr_unreachable_nodes:
    unique_patch_nodes.add((i,j))

print("Number of unique patches - for unreachable nodes",len(unique_patch_nodes))

unique_patch_terminals = set()

for i,j,_,_ in terminals:
    unique_patch_terminals.add((i,j))


In [ ]:
@jit(nopython=True)
def get_neighbors_fast(i, j, x, y, R, shape, patch_size):
    n, m = shape
    X_L = x + (i * patch_size)
    Y_L = y + (j * patch_size)
    neighbors = []
    for dx in range(-R, R+1):
        for dy in range(-R, R+1):
            if dx == 0 and dy == 0:
                continue
            nx, ny = X_L + dx, Y_L + dy
            if 0 <= nx < n and 0 <= ny < m:
                p_i, p_j = nx // patch_size, ny // patch_size
                p_x, p_y = nx % patch_size, ny % patch_size
                neighbors.append((p_i, p_j, p_x, p_y))
    return neighbors

@jit(nopython=True)
def process_edge_points(u, w, X_r, R, shape, patch_size, th):
    new_points = []
    for edge_point in u:
        i, j, x, y = edge_point
        neighbors = get_neighbors_fast(i, j, x, y, R, shape, patch_size)
        for neighbor in neighbors:
            i, j, x, y = neighbor
            if w[i, j, x, y] > th and X_r[i, j, x, y] == 0:
                X_r[i, j, x, y] = int(1 / w[i, j, x, y])
                new_points.append((i, j, x, y))
    return new_points, X_r




f = np.array(predictions)
f = np.squeeze(f,axis=-1)
w = f.copy()

X_r = full_h_patches.copy()
u = terminals.copy()
X = full_h_patches.copy()
r_th = 0.1
new_points = []

new_points, X_r = process_edge_points(u, w, X_r, R, full_h.shape, patch_size, r_th)

print(X_r.shape)

# X_r[X_r>=1] = int(1/.9)
X_r[X_r==0] = large_val

In [ ]:
def add_vector_to_point(point, vector):
    return (point[0] + vector[0], point[1] + vector[1])
def points_within_radius(center, radius, points):
    # Convert center to a numpy array
    center = np.array(center)
    
    # Calculate the squared distances from each point in the array to the center
    distances = np.sqrt(np.sum((points - center) ** 2, axis=1))
    
    # Find points where the distance is less than or equal to the radius
    within_radius = points[distances <= radius]
    
    return within_radius

def find_edge_points(full_h_patches,data):
    data_set = set(tuple(point) for point in data)  # Convert array to a set of tuples for fast lookup
    visited = set()  # To track visited points
    edgepoints = []  # List to store endpoints
    p_i,p_j,p_rows,p_columns = full_h_patches.shape

    # Directions corresponding to N, NE, E, SE, S, SW, W, NW
    directions = [(0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1)]

    for point in data:
        neighbors = []
        point_tuple = tuple(point)
        i,j,x,y = point_tuple
        if point_tuple not in visited:
            # Check all 8 directions and count neighbors
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                n_i,n_j= i,j

                if 0>nx or nx>=p_rows:
                    n_i = i+dx
                    nx = 511 if nx == -1 else 0 if nx == 512 else nx

                if 0>ny or ny>=p_columns:
                    n_j = j+dy
                    ny = 511 if ny == -1 else 0 if ny == 512 else ny
                    
                if (n_i,n_j,nx, ny) in data_set:
                    neighbors.append((n_i,n_j,nx, ny))

            num_neighbors = len(neighbors)
            
            if num_neighbors < 8:
                edgepoints.append(point)  # Zero or one neighbor, so it's an endpoint
                
            # Mark point and its neighbors as visited
            visited.add(point_tuple)
            # visited.update(neighbors)

    return np.array(edgepoints),visited

def find_source_terminal_pair(source_terms,X_L,Y_L,i,j,radius):
    n_reachable_terms = []
    directions = [(0,0),(0, 1), (0, -1), (1, 0), (-1, 0),
     (-1, -1), (1, -1), (1, 1), (-1, 1)]
    
    resulting_points = np.array([(i, j)]) + directions
    mask = np.isin(source_terms[:, :2], resulting_points).all(axis=1)
    reachable_terms = source_terms[mask, 2:] + source_terms[mask, :2] * 512
    
    if(len(reachable_terms)!=0):
        n_reachable_terms = points_within_radius((X_L,Y_L), radius, reachable_terms)
    return n_reachable_terms

data = np.array(water_index)
water_edges,visited = find_edge_points(gt_patches,data)
a = reachable_nodes.copy()
a = np.array(list(a))
source_terms = np.concatenate((a,water_edges))
source_terminal_pair = {}
tot_sp = 0
for terminal in terminals:
    i,j,x,y = terminal
    X_L = (x) + (i*512) 
    Y_L = (y) + (j*512)
    pair_val = find_source_terminal_pair(source_terms,X_L,Y_L,i,j,R)
    tot_sp += len(pair_val)
    source_terminal_pair[f'{(X_L,Y_L)}'] = (pair_val)




# # Save using Pickle
# file_path = get_source_terminal_pair_path(years)
# with open(file_path, 'wb') as file:
#     pickle.dump(source_terminal_pair, file)

# # Load the dictionary back
# with open(file_path, 'rb') as file:
#     source_terminal_pair = pickle.load(file)
#     print(len(source_terminal_pair))

print("Number of Source Terminal Pair", tot_sp)

In [ ]:
paths = []
terminal_paths = {}
netwk = unpatchify(X_r, full_h.shape)
cnt = 0
for terminal in terminals:
    i,j,x,y = terminal
    X_L = (x) + (i*512) 
    Y_L = (y) + (j*512)
    t_points = (X_L,Y_L)
    source_points = source_terminal_pair[f'{(X_L,Y_L)}']
    goals = list(map(tuple, source_points))
    if len(goals)!= 0:
        netwk_t = create_subgraph_around_terminal(netwk,t_points, 110)
        best_source, shortest_distance, best_path = find_shortest_path(netwk_t,t_points,goals)
        if best_source != None:
            cnt = cnt+1
            terminal_paths[f'{t_points}'] = best_path
            paths.extend(best_path)

print('terminals got connected',cnt)
file_path = generated_paths_from_terminals(years)
with open(file_path, 'wb') as file:
    pickle.dump(terminal_paths, file)

    
    

In [ ]:
predictions = np.array(predictions)

In [ ]:
reachable_canal = np.load('/scratch/gza5dr/Current_Canal_Experiments/Canal_Detection_Experiments/Proposed_Model_Pipeline/implementation/framework/final_framework/output_set3/R110_th0.5_rth0.01_4dil_resnet/reachable_nodes_H_it0_110_0.5_0.01_combined.npy')

In [ ]:
unreachable_canal = np.load('/scratch/gza5dr/Current_Canal_Experiments/Canal_Detection_Experiments/Proposed_Model_Pipeline/implementation/framework/final_framework/output_set3/R110_th0.5_rth0.01_4dil_resnet/unreachable_nodes_H_it0_110_0.5_0.01_combined.npy')

In [ ]:
unreachable_canal

In [ ]:
result = reachable_canal[(reachable_canal[:, 0] == 9) & (reachable_canal[:, 1] == 10)]
result

In [ ]:
import matplotlib.pyplot as plt
patch_size = 512
def visulization(display_list, figsize=(15, 10)):
    _, axes = plt.subplots(nrows=1, ncols=len(display_list), figsize=figsize)
    for i in range(len(display_list)):
            axes[i].imshow(display_list[i])
            axes[i].axis('off') 
    plt.show()
    

for terms in list(terminal_paths.keys()):
    coordinate_tuple = eval(terms)
    X_L,Y_L = coordinate_tuple
    path = terminal_paths[terms]
    print(len(path))
    x,y,p_x,p_y = (int(X_L/patch_size),int(Y_L/patch_size),X_L%patch_size,Y_L%patch_size)
    ck2=full_h_patches[x,y,:,:].copy()
    ck1=full_h_patches[x,y,:,:].copy()
    bp = set()
    for a,b,c in path:
        bp.add((a,b))
    for k,l in bp:
        _,_,k,l = (int(k/patch_size),int(l/patch_size),k%patch_size,l%patch_size)
        cv2.circle(ck2, (l,k), radius=2, color=(6, 0, 0), thickness=3)
    cv2.circle(ck2, (p_y,p_x), radius=5, color=(7, 0, 0), thickness=5)
    result = reachable_canal[(reachable_canal[:, 0] == x) & (reachable_canal[:, 1] == y)]
    result2 = unreachable_canal[(unreachable_canal[:, 0] == x) & (unreachable_canal[:, 1] == y)]
    for _,_,k,l in result:
        cv2.circle(ck1, (l,k), radius=2, color=(8, 0, 0), thickness=5)
        cv2.circle(ck2, (l,k), radius=2, color=(8, 0, 0), thickness=5)
    for _,_,k,l in result2:
        cv2.circle(ck1, (l,k), radius=2, color=(12, 0, 0), thickness=5)
        cv2.circle(ck2, (l,k), radius=2, color=(12, 0, 0), thickness=5)
    p = predictions[x,y,:,:]
    # p[p>0.01]=1
    # visulization([image_patches[x,y,0,:,:],gt_patches[x,y,:,:],predictions[x,y,:,:],ck1,ck2])
    visulization([image_patches[x,y,0,:,:],predictions[x,y,:,:],ck1,ck2])

#### Network Check

In [ ]:
from network_utils import *

def find_isolated_points(data):
    data_set = set(tuple(point) for point in data)  # Convert array to a set of tuples for fast lookup
    visited = set()  # To track visited points
    endpoints = []  # List to store endpoints
    p_i,p_j,p_rows,p_columns = full_h_patches.shape

    # Directions corresponding to N, NE, E, SE, S, SW, W, NW
    directions = [(0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1)]

    for point in data:
        neighbors = []
        point_tuple = tuple(point)
        i,j,x,y = point_tuple
        if point_tuple not in visited:
            # Check all 8 directions and count neighbors
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                n_i,n_j= i,j

                if 0>nx or nx>=p_rows:
                    n_i = i+dx
                    nx = 511 if nx == -1 else 0 if nx == 512 else nx

                if 0>ny or ny>=p_columns:
                    n_j = j+dy
                    ny = 511 if ny == -1 else 0 if ny == 512 else ny
                    
                if (n_i,n_j,nx, ny) in data_set:
                    neighbors.append((n_i,n_j,nx, ny))

            num_neighbors = len(neighbors)
            
            if num_neighbors <= 1:
                endpoints.append(point)  # Zero or one neighbor, so it's an endpoint
                
            # Mark point and its neighbors as visited
            visited.add(point_tuple)
            # visited.update(neighbors)

    return np.array(endpoints),visited
    
def find_common_rows(a, b):
    # Convert b to a set of tuples for faster lookup
    b_set = set(map(tuple, b))

    # Use a list comprehension to find common rows while preserving order
    common = [[a,b,c,d] for a,b,c,d in a if tuple([c+(a*512),d+(b*512)]) in b_set]

    return np.array(common)

for i in range(4):
    print(years[i])
    year = years[i]
    image_patches = get_img_patches(year)
    gt_patches = get_gt_patches(year)
    predictions = get_predictions(year)
    X_r = np.load(get_X_r_path(year))
    full_h = get_full_h(year)
    full_h_patches = get_full_h_patches(year)
    terminals = get_terminals(year)
    n_all_mask = full_h.copy()
    added_nodes = 0
    file_path = generated_paths_from_terminals(year)
    with open(file_path, 'rb') as file:
        terminal_paths = pickle.load(file)
    paths = []
    for terms in list(terminal_paths.keys()):
        paths.extend(terminal_paths[terms])
    for x,y,_ in paths:
        if n_all_mask[x,y]!=1:
            added_nodes += 1
        n_all_mask[x,y] = 1
    print(added_nodes)
    water_index = get_water_source_index(n_all_mask)
    directly_connected_full,not_connected_full = directly_connected_1s(n_all_mask,water_index)
    reachable_nodes, unreachable_nodes = bfs(n_all_mask, directly_connected_full, directly_connected_full,not_connected_full)
    print(f"Number of reachable Canal pixels:{len(reachable_nodes)}")
    
    
    unreachable_nodes_N = np.where(unreachable_nodes == 1)
    d_row_indices, d_col_indices = unreachable_nodes_N
    unreachable_nodes_index_pairs = list(zip(d_row_indices, d_col_indices))
    unreachable_nodes_index_pairs = np.array(unreachable_nodes_index_pairs)
    np.save(f"{outputh_path}/it_1{y}.npy",n_all_mask)
    curr_unreachable_nodes = np.load(get_curr_unreachable_nodes_path(year))
    print(f"Current Number of unreachable Canal pixels:{len(unreachable_nodes_index_pairs)}")
    
    
    


    b = np.array(unreachable_nodes_index_pairs)
    a = np.array(curr_unreachable_nodes)
    c = np.load(get_unreachable_nodes_path(years[i]))
    
    print(f"Prev Number of unreachable Canal pixels:{len(curr_unreachable_nodes)}")

    result = find_common_rows(a, b)
    result2 = find_common_rows(a, c)
    print(f"Unreachable Canal pixels Left in main GT:{len(result)}")
    print(f"Unreachable Canal pixels Left:{len(result)}")
    np.save(f"{outputh_path}/it_1_{year}_mask.npy",n_all_mask)
    
    
    data = np.array(b)
    terminals,visited = find_isolated_points(data)
    # print("Endpoints:", terminals)
    print(len(terminals)
          
    unique_patch_nodes = set()

    for i,j,_,_ in curr_unreachable_nodes:
        unique_patch_nodes.add((i,j))

    print(len(unique_patch_nodes))

    unique_patch_terminals = set()

    for i,j,_,_ in terminals:
        unique_patch_terminals.add((i,j))

    print(len(unique_patch_terminals))

In [ ]:
import numpy as np
from patchify import patchify, unpatchify

p_s = 512

outputh_path = '/scratch/gza5dr/Current_Canal_Experiments/Canal_Detection_Experiments/Proposed_Model_Pipeline/implementation/framework/output_v1_it1'
curr_unreachable_nodes1 = result
# curr_unreachable_nodes=np.load('/scratch/gza5dr/Current_Canal_Experiments/Canal_Detection_Experiments/Proposed_Model_Pipeline/implementation/framework/files/curr_unreachable_nodes.npy')
full_h = n_all_mask
full_h_patches = patchify(full_h,(p_s,p_s),step=p_s)
ground_truth = np.load(f'/scratch/gza5dr/Canal_Datasets/full_data/masks/2021.npy')
cut_1,cut_2 = ground_truth.shape[0]%p_s,ground_truth.shape[1]%p_s
ground_truth = ground_truth[cut_1:,cut_2:]




# Example usage with your data
data = np.array(curr_unreachable_nodes1)

terminals,visited = find_isolated_points(data)
# print("Endpoints:", terminals)
print(len(terminals))


unique_patch_nodes = set()

for i,j,_,_ in curr_unreachable_nodes:
    unique_patch_nodes.add((i,j))
    
print(len(unique_patch_nodes))

unique_patch_terminals = set()

for i,j,_,_ in terminals:
    unique_patch_terminals.add((i,j))
    
print(len(unique_patch_terminals))

In [ ]:
print(f"Number of unreachable Canal pixels:{len(unreachable_nodes)}")

In [ ]:
unreachable_nodes_N = np.where(unreachable_nodes == 1)
d_row_indices, d_col_indices = unreachable_nodes_N
unreachable_nodes_index_pairs = list(zip(d_row_indices, d_col_indices))
unreachable_nodes_index_pairs = np.array(unreachable_nodes_index_pairs)

In [ ]:
unreachable_nodes_index_pairs.shape

In [ ]:
prev = 132224
current = len(unreachable_nodes)
print(prev-current)

In [ ]:
unreachable_nodes_index_pairs.shape

In [ ]:
np.save(f"{outputh_path}/it_1.npy",n_all_mask)

In [ ]:
np.save(f"{outputh_path}/it_1_unreachable.npy",n_all_mask)

In [ ]:
curr_unreachable_nodes = np.load(f'{outputh_path}/curr_unreachable_nodes.npy')

In [ ]:
def find_common_rows(a, b):
    # Convert b to a set of tuples for faster lookup
    b_set = set(map(tuple, b))
    
    # Use a list comprehension to find common rows while preserving order
    common = [[a,b,c,d] for a,b,c,d in a if tuple([c+(a*512),d+(b*512)]) in b_set]
    
    return np.array(common)

b = np.array(unreachable_nodes_index_pairs)
a = np.array(curr_unreachable_nodes)

result = find_common_rows(a, b)
print(len(result))

In [ ]:
result

In [ ]:
def find_common_rows(a_s, b_s):
    # Convert b to a set of tuples for faster lookup
    
    c_set = [[m, n] for m, n, o, p in b_s]
    c_set = set(map(tuple, c_set))
    
    # Use a list comprehension to find common rows while preserving order
    common = [[a,b,c,d] for a,b,c,d in a_s if tuple([a,b]) in c_set]
    
    return np.array(common)

b = np.array(result)
a = np.array(curr_unreachable_nodes)

result = find_common_rows(a, b)
print(len(result))

In [ ]:
result[:]